# Quality Control (QC)

The KPF DRP has several Quality Control (QC) methods that can be run on the L0, 2D, L1, and 2D objects.  The usual application of QC tests is executing them during normal processing in the main recipe.  The QC methods are defined in the ``QCDefinitions`` class in ``modules/quality_control/src/quality_control.py``.  The results of these QC checks are added to the primary headers of kpf objects, which are written to 2D, L1, and L2 FITS files (but not the L0 files, which, with rare exceptions, are not modified after data collection at WMKO).  The FITS header keywords for QC tests produce are defined in :doc:`data_format`.  One can find results from the QC tests from primary header keywords on the command line using a command like 

``fitsheader -e 0 /data/kpf/L2/20230701/KP.20230701.49940.99_L2.fits | grep QC:``

The KPF DRP has several Quality Control (QC) methods that can be run on the L0, 2D, L1, and 2D objects. The QC tests are run during normal processing in the main recipe. The results of these QC checks are added to the primary headers of kpf objects, which are written to 2D, L1, and L2 FITS files (but not the L0 files, which with rare exceptions are not modified after data collection at WMKO). 

One can also produce a 'Quality Control Report' for a given observation.  The example below is for an Etalon exposure.

In [1]:
import os
from modules.Utils.kpf_parse import get_kpf_data, HeaderParse
import modules.quality_control.src.quality_control as qc
from modules.quality_control.src.quality_control import execute_all_QCs
from IPython.display import display, HTML
from modules.Utils.kpf_parse import get_datecode
from kpfpipe.models.level2 import KPF2
from modules.quality_control.src.quality_control import *

ObsID = 'KP.20240426.51689.91' # Etalon exposure
#ObsID = 'KP.20240424.25862.05' # Star
L2_file = '/data/L2/' + get_datecode(ObsID) + '/' + ObsID + '_L2.fits'
L2 = KPF2.from_fits(L2_file)
QC_report(L2, print_output=True) # print results

Quality Control Report for KP.20240426.51689.91.fits (Etalon)

All QC Keywords:
    Keyword      Level  QC Test Description
    NOTJUNK ✓    L0     File is not in list of junk files
    DATAPRL0 ✓   L0     Expected L0 data products present with non-zero array sizes
    KWRDPRL0 ✓   L0     Expected L0 header keywords present
    TIMCHKL0 ✓   L0     Timing consistency in L0 header keywords and ExpMeter table
    GOODREAD ✓   L0     CCD read properly (Texp !≈ 6 sec and Texp_desired > 7 sec)
    NTPGOOD ✓    L0     NTP time accurate to within 100 ms
    DATAPR2D ✓   2D     Green and Red CCD data present with expected array sizes
    POS2DSNR ✓   2D     Green and Red CCD data/variance^0.5 not significantly negative
    OLDBIAS ✓    2D     Master bias from within 5 days of this observation
    OLDDARK ✓    2D     Master dark from within 5 days of this observation
    OLDFLAT ✓    2D     Master flat from within 5 days of this observation
    MONOTWLS ✓   L1     Wavelength solution is monotoni

The QC methods are defined in the `QCDefinitions` class in `modules/quality_control/src/quality_control.py`.

In [2]:
myQCdef = qc.QCDefinitions()
QC_lists = myQCdef.list_qc_metrics()

# prevent scrolling in the resulting Notebook cell
display(HTML("<style>.output_scroll { height: auto !important; }</style>"))

Quality Control tests for L0:
   Name: not_junk
      Description: File is not in list of junk files
      Date levels: ['L0', '2D', 'L1', 'L2']
      Date type: int
      Required data products: []
      Spectrum types (applied to): ['all']
      Master types (required for): ['all']
      Drift types (required for): ['all']
      Keyword: NOTJUNK
      Keyword fail value: 0
      Comment: QC: Not in list of junk files
      Database column: None

   Name: L0_data_products
      Description: Expected L0 data products present with non-zero array sizes
      Date levels: ['L0']
      Date type: int
      Required data products: []
      Spectrum types (applied to): ['all']
      Master types (required for): ['all']
      Drift types (required for): ['all']
      Keyword: DATAPRL0
      Keyword fail value: 0
      Comment: QC: L0 data present
      Database column: None

   Name: L0_header_keywords_present
      Description: Expected L0 header keywords present
      Date levels: ['L0']
  

The method `test_all_QCs(kpf_object, data_level)` runs the QC tests on a kpf_object for the stated data_level.

In [3]:
ObsID = 'KP.20230701.49940.99' # 185144 (stellar observation)
data_level = 'L2'
if os.path.isfile(get_kpf_data(ObsID, data_level, return_kpf_object=False)):
    print(f'\033[1m{data_level} QC tests on {ObsID}\033[0m')
    print(f'\033[1m***********************************\033[0m')
    kpf_object = get_kpf_data(ObsID, data_level)
    junk = execute_all_QCs(kpf_object, data_level)

L2 QC tests on KP.20230701.49940.99
***********************************
INFO: Spectrum type: Star
INFO: QC: not_junk (NOTJUNK; File is not in list of junk files)
INFO: QC result: True (True = pass)
INFO: QC: L2_datetime (TIMCHKL2; Timing consistency in L2 files)
INFO: QC result: True (True = pass)
INFO: QC: data_L2 (DATAPRL2; All data present in L2)
INFO: QC result: True (True = pass)
INFO: QC: add_kpfera (KPFERA; Not a QC test; KPFERA keyword added to header)
INFO: Result: KPFERA=1.0


Now loop over data levels and obserations

In [4]:
# Define data levels to loop over
data_levels = ['L0', '2D', 'L1', 'L2']

# Define list of observations to loop over - one of every spectrum type
ObsIDs = [
          'KP.20230701.49940.99', # 185144 (stellar observation)
          'KP.20240525.77699.94', # Socal
          'KP.20240528.04120.26', # autocal-bias
          'KP.20240528.08502.04', # autocal-dark
          'KP.20240527.84455.02', # autocal-flat-all
          'KP.20240526.11989.38', # autocal-lfc-all-eve
          'KP.20240528.07447.61', # autocal-thar-all-eve
          'KP.20240527.11183.00', # autocal-une-all-eve
          'KP.20240528.05681.26', # autocal-etalon-all-eve
          'KP.20240527.51851.54', # autocal-etalon-all-night
          'KP.20240528.07772.51', # autocal-thar-hk
         ]

# Run QC tests
for ObsID in ObsIDs:
    for data_level in data_levels:
        if os.path.isfile(get_kpf_data(ObsID, data_level, return_kpf_object=False)):
            print(f'\033[1m{data_level} QC tests on {ObsID}\033[0m')
            print(f'\033[1m***********************************\033[0m')
            kpf_object = get_kpf_data(ObsID, data_level)
            junk = execute_all_QCs(kpf_object, data_level)
            print()
            
# prevent scrolling in the resulting Notebook cell
display(HTML("<style>.output_scroll { height: auto !important; }</style>"))

L0 QC tests on KP.20230701.49940.99
***********************************
INFO: Spectrum type: Star
INFO: QC: not_junk (NOTJUNK; File is not in list of junk files)
INFO: QC result: True (True = pass)
INFO: QC: L0_data_products (DATAPRL0; Expected L0 data products present with non-zero array sizes)
INFO: QC result: True (True = pass)
INFO: QC: L0_header_keywords_present (KWRDPRL0; Expected L0 header keywords present)
INFO: QC result: True (True = pass)
INFO: QC: L0_datetime (TIMCHKL0; Timing consistency in L0 header keywords and ExpMeter table)
INFO: QC result: True (True = pass)
INFO: QC: EM_not_saturated (EMSAT; 2+ reduced EM pixels within 90% of saturation in EM-SCI or EM-SKY)
INFO: QC result: True (True = pass)
INFO: QC: EM_flux_not_negative (EMNEG; Negative flux in the EM-SCI and EM-SKY by looking for 20 consecuitive pixels in the summed spectra with negative flux.)
INFO: QC result: True (True = pass)
INFO: QC: add_kpfera (KPFERA; Not a QC test; KPFERA keyword added to header)
INFO: 

INFO: Spectrum type: Sun
INFO: QC: not_junk (NOTJUNK; File is not in list of junk files)
INFO: QC result: True (True = pass)
INFO: QC: L0_data_products (DATAPRL0; Expected L0 data products present with non-zero array sizes)
INFO: QC result: True (True = pass)
INFO: QC: L0_header_keywords_present (KWRDPRL0; Expected L0 header keywords present)
INFO: QC result: True (True = pass)
INFO: QC: L0_datetime (TIMCHKL0; Timing consistency in L0 header keywords and ExpMeter table)
INFO: QC result: True (True = pass)
INFO: Not running QC: EM_not_saturated (2+ reduced EM pixels within 90% of saturation in EM-SCI or EM-SKY) because ['ExpMeter'] not in list of expected data products(['Telemetry', 'Green', 'Red', 'Pyrheliometer'])
INFO: Not running QC: EM_flux_not_negative (Negative flux in the EM-SCI and EM-SKY by looking for 20 consecuitive pixels in the summed spectra with negative flux.) because ['ExpMeter'] not in list of expected data products(['Telemetry', 'Green', 'Red', 'Pyrheliometer'])
INFO

INFO: Spectrum type: LFC
INFO: Running QC: not_junk (NOTJUNK; File is not in list of junk files.)
INFO: QC result: True (True = pass)
INFO: Running QC: L0_data_products (DATAPRL0; Expected L0 data products present with non-zero array sizes.)
INFO: Data products expected in this L0 file: ['Green', 'Red', 'Ca_HK', 'Telemetry']
INFO: Data products in L0 file: ['Green', 'Red', 'HK', 'Telemetry']
INFO: Possible data products in L0 file: ['Green', 'Red', 'CaHK', 'ExpMeter', 'Guider', 'Telemetry', 'Pyrheliometer']
INFO: QC result: True (True = pass)
INFO: Running QC: L0_header_keywords_present (KWRDPRL0; Expected L0 header keywords present.)
INFO: QC result: True (True = pass)
INFO: Running QC: L0_datetime (TIMCHKL0; Timing consistency in L0 header keywords and ExpMeter table.)
INFO: QC result: True (True = pass)
INFO: Not running QC: EM_not_saturated (2+ reduced EM pixels within 90% of saturation in EM-SCI or EM-SKY.) because ['ExpMeter'] not in list of expected data products(['Telemetry', '

INFO: Spectrum type: ThAr
INFO: Running QC: not_junk (NOTJUNK; File is not in list of junk files.)
INFO: QC result: True (True = pass)
INFO: Not running QC: EM_not_saturated (2+ reduced EM pixels within 90% of saturation in EM-SCI or EM-SKY.) because ['ExpMeter'] not in list of expected data products(['Telemetry', 'Config', 'Receipt', 'Green', 'Red', 'CaHK'])
INFO: Not running QC: EM_flux_not_negative (Negative flux in the EM-SCI and EM-SKY by looking for 20 consecuitive pixels in the summed spectra with negative flux.) because ['ExpMeter'] not in list of expected data products(['Telemetry', 'Config', 'Receipt', 'Green', 'Red', 'CaHK'])
INFO: Not running QC: D2_lfc_flux (LFC frame that goes into a master has sufficient flux) because ThAr not in list of spectrum types: ['LFC']
INFO: Not running QC: data_2D_bias_low_flux (Flux is low in bias exposure.) because ThAr not in list of spectrum types: ['Bias']
INFO: Not running QC: data_2D_dark_low_flux (Flux is low in dark exposure.) because 

INFO: QC result: True (True = pass)
INFO: Running QC: add_kpfera (KPFERA; Not a QC test; used to add the KPFERA keyword to header.)
INFO: The datetime of ObsID is 2024-05-27 03:06:23.
INFO: Result: KPFERA=2.0
INFO: Running QC: L0_bad_readout_check (GOODREAD; Check Texp that identifies error in reading CCD)
INFO: Method L0_bad_readout_check does not exist in qc_obj or another AttributeError occurred: 'QC2D' object has no attribute 'L0_bad_readout_check'
INFO: Running QC: D2_master_bias_age (OLDBIAS; Check master dark file age)
INFO: QC result: True (True = pass)
INFO: Running QC: D2_master_dark_age (OLDDARK; Check master dark file age)
INFO: QC result: True (True = pass)
INFO: Running QC: D2_master_flat_age (OLDFLAT; Check master flat file age)
INFO: QC result: True (True = pass)

L1 QC tests on KP.20240527.11183.00
***********************************
INFO: Spectrum type: UNe
INFO: Running QC: not_junk (NOTJUNK; File is not in list of junk files.)
INFO: QC result: True (True = pass)
INF

INFO: QC result: True (True = pass)
INFO: Running QC: D2_master_dark_age (OLDDARK; Check master dark file age)
INFO: QC result: True (True = pass)
INFO: Running QC: D2_master_flat_age (OLDFLAT; Check master flat file age)
INFO: QC result: True (True = pass)

L1 QC tests on KP.20240528.05681.26
***********************************
INFO: Spectrum type: Etalon
INFO: Running QC: not_junk (NOTJUNK; File is not in list of junk files.)
INFO: QC result: True (True = pass)
INFO: Running QC: monotonic_wavelength_solution (MONOTWLS; Wavelength solution is monotonic.)
INFO: QC result: True (True = pass)
INFO: Running QC: data_L1_red_green (DATAPRL1; Red/Green data present in L1 with expected shapes.)
INFO: QC result: True (True = pass)
INFO: Running QC: data_L1_CaHK (CAHKPRL1; CaHK data present in L1 with expected shape.)
INFO: QC result: True (True = pass)
INFO: Running QC: add_kpfera (KPFERA; Not a QC test; used to add the KPFERA keyword to header.)
INFO: The datetime of ObsID is 2024-05-28 01:34

INFO: Spectrum type: Etalon
INFO: Running QC: not_junk (NOTJUNK; File is not in list of junk files.)
INFO: QC result: True (True = pass)
INFO: Running QC: monotonic_wavelength_solution (MONOTWLS; Wavelength solution is monotonic.)
INFO: QC result: True (True = pass)
INFO: Running QC: data_L1_red_green (DATAPRL1; Red/Green data present in L1 with expected shapes.)
INFO: QC result: True (True = pass)
INFO: Running QC: data_L1_CaHK (CAHKPRL1; CaHK data present in L1 with expected shape.)
INFO: QC result: True (True = pass)
INFO: Running QC: add_kpfera (KPFERA; Not a QC test; used to add the KPFERA keyword to header.)
INFO: The datetime of ObsID is 2024-05-27 14:24:11.
INFO: Result: KPFERA=2.0
INFO: Not running QC: L1_check_snr_lfc (Check for saturated LFC frames.) because Etalon not in list of spectrum types: ['LFC']
INFO: Running QC: L1_correct_wls_check (WLSL1; Check WLS files used by L1 file)
INFO: QC result: False (True = pass)
INFO: Running QC: L1_WLSFILE_age (OLDWLS; Check WLSFILE f

INFO: QC result: False (True = pass)
INFO: Running QC: L1_WLSFILE_age (OLDWLS; Check WLSFILE file age)
INFO: QC result: True (True = pass)
INFO: Running QC: L1_WLSFILE2_age (OLDWLS2; Check WLSFILE2 file age)
INFO: QC result: True (True = pass)
INFO: Not running QC: L1_FLAT_SNR (Check SNR of flat) because  not in list of spectrum types: ['Flat']
INFO: Not running QC: L1_LFC_lines (Check number and distribution of LFC lines/order) because  not in list of spectrum types: ['LFC']
INFO: Not running QC: L1_Etalon_lines (Check number and distribution of Etalon lines/order) because  not in list of spectrum types: ['Etalon']

L2 QC tests on KP.20240528.07772.51
***********************************
INFO: Spectrum type: 
INFO: Running QC: not_junk (NOTJUNK; File is not in list of junk files.)
INFO: QC result: True (True = pass)
INFO: Running QC: L2_datetime (TIMCHKL2; Timing consistency in L2 files.)
INFO: QC result: True (True = pass)
INFO: Running QC: data_L2 (DATAPRL2; All data present in L2.)
